In [6]:
# 单元格 1: 导入
import os
import shutil
import torch
import scanpy as sc
import anndata
import webdataset as wds
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import io
from tqdm.auto import tqdm

# 关键：导入PyTorch Geometric
import torch.utils.data
from torch_geometric.data import Data as PyGData
from torch_geometric.loader import DataLoader as PyGDataLoader # 或者直接用torch的Dataloader + PyG的collate_fn
from torch_geometric.data import Batch as PyGBatch

# 从您准备要修改的open-clip目录中导入一些辅助类
# 如果您的notebook和open-clip库不在同一个环境下，您可能需要调整sys.path
# from open_clip_train.data import DataInfo, SharedEpoch 

# 为了让Notebook自包含，我们在这里重新定义这些简单的辅助类
from dataclasses import dataclass
@dataclass
class DataInfo:
    dataloader: torch.utils.data.DataLoader

In [7]:
# 单元格 2: 创建模拟数据和文件结构

# --- 配置 ---
TEST_ENV_DIR = "/cwStorage/nodecw_group/jijh/openclip_train/spaglam_test_environment"
SHARDS_DIR = os.path.join(TEST_ENV_DIR, "shards")
METADATA_DIR = os.path.join(TEST_ENV_DIR, "metadata")
DUMMY_ADATA_PATH = os.path.join(METADATA_DIR, "dummy_master_adata.h5ad")
DUMMY_SHARD_PATH_PATTERN = os.path.join(SHARDS_DIR, "shard-%06d.tar")

NUM_SPOTS = 100
NUM_NEIGHBORS = 6
IMAGE_SIZE = (224, 224)

# --- 清理并创建目录 ---
if os.path.exists(TEST_ENV_DIR):
    shutil.rmtree(TEST_ENV_DIR)
os.makedirs(SHARDS_DIR, exist_ok=True)
os.makedirs(METADATA_DIR, exist_ok=True)
print(f"✅ 测试环境目录 '{TEST_ENV_DIR}' 已创建。")

# --- 1. 生成模拟AnnData文件 ---
print("\n🛠️ 正在生成模拟 AnnData 文件...")
spot_ids = [f"DUMMY_SPOT_{i}" for i in range(NUM_SPOTS)]
adata = anndata.AnnData(X=np.random.rand(NUM_SPOTS, 10)) # 伪基因表达数据
adata.obs_names = spot_ids
adata.obs['sample_id'] = 'DUMMY_SAMPLE'
# 为每个spot生成指向即将创建的文件的路径
adata.obs['image_path'] = [os.path.join(SHARDS_DIR, f"{sid}.png") for sid in spot_ids]
adata.obs['sentence_path'] = [os.path.join(SHARDS_DIR, f"{sid}.txt") for sid in spot_ids]
adata.obsm['spatial'] = np.random.rand(NUM_SPOTS, 2) * 1000 # 伪空间坐标

# 创建一个简单的k-NN图
from sklearn.neighbors import kneighbors_graph
connectivity_matrix = kneighbors_graph(adata.obsm['spatial'], NUM_NEIGHBORS, mode='connectivity', include_self=False)
adata.obsp['spatial_connectivities'] = connectivity_matrix
adata.write_h5ad(DUMMY_ADATA_PATH)
print(f"✅ 模拟 AnnData 文件已保存至: {DUMMY_ADATA_PATH}")

# --- 2. 生成模拟WebDataset分片 ---
print("\n🛠️ 正在生成模拟 WebDataset 分片...")
shard_path = DUMMY_SHARD_PATH_PATTERN % 0
with wds.TarWriter(shard_path) as sink:
    for spot_id in tqdm(spot_ids, desc="打包模拟数据"):
        # 创建一个带文字的伪图像
        img = Image.new('RGB', IMAGE_SIZE, color = 'black')
        draw = ImageDraw.Draw(img)
        try:
            # 尝试加载一个字体，如果失败则使用默认
            font = ImageFont.truetype("arial.ttf", 15)
        except IOError:
            font = ImageFont.load_default()
        draw.text((10, 10), spot_id, fill='white', font=font)
        
        # 将图像保存到内存中的字节流
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='PNG')
        image_data = img_byte_arr.getvalue()

        # 创建伪基因语句
        sentence_data = f"GENES FOR {spot_id}: GENE_A GENE_B GENE_C".encode('utf-8')
        
        # 写入样本
        sample = {"__key__": spot_id, "png": image_data, "txt": sentence_data}
        sink.write(sample)
        
        # 同时保存伪PNG和TXT文件，供AnnData中的路径引用
        with open(os.path.join(SHARDS_DIR, f"{spot_id}.png"), "wb") as f_img: f_img.write(image_data)
        with open(os.path.join(SHARDS_DIR, f"{spot_id}.txt"), "wb") as f_txt: f_txt.write(sentence_data)

print(f"✅ 模拟 WebDataset 分片已保存至: {shard_path}")

✅ 测试环境目录 '/cwStorage/nodecw_group/jijh/openclip_train/spaglam_test_environment' 已创建。

🛠️ 正在生成模拟 AnnData 文件...
✅ 模拟 AnnData 文件已保存至: /cwStorage/nodecw_group/jijh/openclip_train/spaglam_test_environment/metadata/dummy_master_adata.h5ad

🛠️ 正在生成模拟 WebDataset 分片...


打包模拟数据:   0%|          | 0/100 [00:00<?, ?it/s]

✅ 模拟 WebDataset 分片已保存至: /cwStorage/nodecw_group/jijh/openclip_train/spaglam_test_environment/shards/shard-000000.tar


In [8]:
# 单元格 3: 复制并粘贴我们正在测试的核心代码

# -------------------------------------------------------------
# 这是您计划放在 src/open_clip_train/spaglam_data.py 的代码
# -------------------------------------------------------------
import logging

class SpaGLaMWDataset(torch.utils.data.IterableDataset):
    def __init__(self, tar_urls, anndata_path, image_processor, tokenizer):
        super().__init__()
        self.tar_urls = tar_urls
        self.anndata_path = anndata_path
        self.image_processor = image_processor
        self.tokenizer = tokenizer
        self.adata = None

    def _init_worker(self):
        if self.adata is None:
            self.adata = sc.read_h5ad(self.anndata_path, backed='r')

    def _process_sample(self, sample):
        self._init_worker()
        center_key = sample['__key__']
        
        try:
            center_idx = self.adata.obs_names.get_loc(center_key)
            neighbor_indices = self.adata.obsp['spatial_connectivities'][center_idx].indices
            neighbor_keys = self.adata.obs_names[neighbor_indices].tolist()
            
            all_keys = [center_key] + neighbor_keys
            num_nodes = len(all_keys)

            paths_df = self.adata.obs.loc[all_keys, ['image_path', 'sentence_path']]
            
            images, sentences = [], []
            for key in all_keys:
                if key == center_key:
                    img = Image.open(io.BytesIO(sample['png'])).convert("RGB")
                    sent = sample['txt'].decode('utf-8')
                else:
                    img_path, sent_path = paths_df.loc[key]
                    try: img = Image.open(img_path).convert("RGB")
                    except (FileNotFoundError, OSError): img = Image.new('RGB', (224, 224), color='grey')
                    try: 
                        with open(sent_path, 'r') as f: sent = f.read()
                    except (FileNotFoundError, OSError): sent = ""
                images.append(self.image_processor(img))
                sentences.append(sent)

            tokenized_texts = self.tokenizer(sentences)
            
            key_to_local_idx = {key: i for i, key in enumerate(all_keys)}
            edge_list = []
            center_local_idx = 0
            
            # 使用AnnData中的邻接矩阵构建局部图
            for i, key in enumerate(all_keys):
                global_idx = self.adata.obs_names.get_loc(key)
                # 获取该节点的邻居的全局索引
                row_indices = self.adata.obsp['spatial_connectivities'][global_idx].indices
                
                for neighbor_global_idx in row_indices:
                    neighbor_key = self.adata.obs_names[neighbor_global_idx]
                    if neighbor_key in key_to_local_idx:
                        # 确保只添加子图内部的边
                        j = key_to_local_idx[neighbor_key]
                        edge_list.append([i, j])

            edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous() if edge_list else torch.empty((2, 0), dtype=torch.long)

            return PyGData(
                x_image=torch.stack(images),
                x_text=tokenized_texts,
                edge_index=edge_index,
                num_nodes=num_nodes,
                center_key=center_key,
            )
        except Exception as e:
            logging.warning(f"Skipping sample {center_key} due to error: {e}")
            return None

    def __iter__(self):
        pipeline = wds.DataPipeline(
            wds.SimpleShardList(self.tar_urls) if not isinstance(self.tar_urls, wds.ShardList) else self.tar_urls,
            wds.split_by_worker,
            wds.tarfile_to_samples(),
            wds.decode("pilrgb", "txt"),
            wds.map(self._process_sample),
            wds.select(lambda x: x is not None),
        )
        return iter(pipeline)

def get_spaglam_dataset(args, preprocess_fn, is_train, epoch=0, tokenizer=None):
    input_shards = args.train_data
    dataset = SpaGLaMWDataset(
        tar_urls=input_shards,
        anndata_path=args.anndata_path,
        image_processor=preprocess_fn,
        tokenizer=tokenizer,
    )
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=args.batch_size,
        num_workers=args.workers,
        collate_fn=PyGBatch.from_data_list,
        drop_last=is_train,
    )
    # 模拟附加元数据
    dataloader.num_samples = args.train_num_samples
    if dataloader.num_samples:
        dataloader.num_batches = dataloader.num_samples // args.batch_size
    else:
        dataloader.num_batches = 0
    return DataInfo(dataloader=dataloader)

# -------------------------------------------------------------
# 这是您计划修改的 src/open_clip_train/data.py 的核心部分
# -------------------------------------------------------------
def get_dataset_fn(data_path, dataset_type):
    if dataset_type == "spaglam":
        return get_spaglam_dataset
    # 在这个测试中我们不关心其他类型
    else:
        raise ValueError(f"Unsupported dataset type for this test: {dataset_type}")

def get_data(args, preprocess_fns, epoch=0, tokenizer=None):
    preprocess_train, preprocess_val = preprocess_fns
    data = {}
    if args.train_data:
        data["train"] = get_dataset_fn(args.train_data, args.dataset_type)(
            args, preprocess_train, is_train=True, epoch=epoch, tokenizer=tokenizer)
    return data

print("✅ 核心数据处理代码已加载到Notebook中。")

✅ 核心数据处理代码已加载到Notebook中。


In [9]:
# 单元格 4: 模拟训练脚本的调用并验证输出

# --- 1. 模拟命令行参数 (args) ---
from argparse import Namespace
args = Namespace(
    dataset_type='spaglam',
    train_data=[DUMMY_SHARD_PATH_PATTERN % 0], # 使用我们创建的模拟分片
    anndata_path=DUMMY_ADATA_PATH,            # 使用我们创建的模拟AnnData
    train_num_samples=NUM_SPOTS,
    batch_size=4,
    workers=0, # 关键：在notebook中设为0，以便于调试，避免多进程问题
    world_size=1,
    distributed=False
)
print("--- 模拟参数 ---")
for k, v in vars(args).items():
    print(f"{k}: {v}")

# --- 2. 模拟预处理器和分词器 ---
from torchvision import transforms
dummy_preprocess_fn = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 一个非常简单的分词器，返回固定长度的随机整数张量
dummy_tokenizer = lambda texts: torch.randint(0, 49408, (len(texts), 77), dtype=torch.long)
print("\n✅ 模拟的预处理器和分词器已准备就绪。")


# --- 3. 调用 get_data 获取 DataLoader ---
print("\n--- 调用 get_data() ---")
data = get_data(args, (dummy_preprocess_fn, None), tokenizer=dummy_tokenizer)
dataloader = data['train'].dataloader
print("✅ 成功创建DataLoader！")

# --- 4. 从DataLoader中取出一个批次并进行最终验证 ---
print("\n--- 最终验证：从DataLoader中获取一个批次 ---")
try:
    batch = next(iter(dataloader))
    
    print("\n✅ [成功] 成功从DataLoader中获取一个批次的数据！")
    print("-" * 50)
    
    # --- 验证 1: 检查批次类型 ---
    print(f"批次类型: {type(batch)}")
    assert isinstance(batch, PyGBatch), "批次类型不正确！应为 torch_geometric.data.Batch"
    print("✅ [类型正确] 批次是 torch_geometric.data.Batch 对象。")
    
    # --- 验证 2: 检查批次内容 ---
    print("\n批次内容概览:")
    print(batch)
    
    # --- 验证 3: 检查关键属性 ---
    print("\n关键属性检查:")
    print(f"批次中的图数量 (应等于batch_size): {batch.num_graphs}")
    assert batch.num_graphs == args.batch_size, "批次中的图数量不等于batch_size"
    
    total_nodes = sum([NUM_NEIGHBORS + 1] * args.batch_size) # 理论上的节点总数
    print(f"批次中的节点总数: {batch.num_nodes}")
    # 注意：由于某些邻居可能缺失，实际节点数可能略小于理论值，这里不作严格断言
    
    print(f"x_image 张量形状: {batch.x_image.shape}")
    assert batch.x_image.shape[0] == batch.num_nodes
    assert tuple(batch.x_image.shape[1:]) == (3, IMAGE_SIZE[0], IMAGE_SIZE[1])
    
    print(f"x_text 张量形状: {batch.x_text.shape}")
    assert batch.x_text.shape[0] == batch.num_nodes
    
    print(f"edge_index 张量形状: {batch.edge_index.shape}")
    
    print(f"batch 索引向量长度 (应等于节点总数): {len(batch.batch)}")
    assert len(batch.batch) == batch.num_nodes
    
    print("\n✅ [内容正确] 批次对象的所有关键属性都符合预期！")
    print("-" * 50)
    print("\n🎉🎉🎉 恭喜！数据加载管道验证成功！您可以放心地将这些代码集成到open-clip项目中了。🎉🎉🎉")

except Exception as e:
    print(f"\n❌ [失败] 在获取或验证批次时发生错误: {e}")
    import traceback
    traceback.print_exc()

--- 模拟参数 ---
dataset_type: spaglam
train_data: ['/cwStorage/nodecw_group/jijh/openclip_train/spaglam_test_environment/shards/shard-000000.tar']
anndata_path: /cwStorage/nodecw_group/jijh/openclip_train/spaglam_test_environment/metadata/dummy_master_adata.h5ad
train_num_samples: 100
batch_size: 4
workers: 0
world_size: 1
distributed: False

✅ 模拟的预处理器和分词器已准备就绪。

--- 调用 get_data() ---
✅ 成功创建DataLoader！

--- 最终验证：从DataLoader中获取一个批次 ---

❌ [失败] 在获取或验证批次时发生错误: module 'webdataset' has no attribute 'ShardList'


Traceback (most recent call last):
  File "/tmp/ipykernel_152893/2261647055.py", line 41, in <module>
    batch = next(iter(dataloader))
                 ~~~~^^^^^^^^^^^^
  File "/public/home/jijh/micromamba/envs/gigapath/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 491, in __iter__
    return self._get_iterator()
           ~~~~~~~~~~~~~~~~~~^^
  File "/public/home/jijh/micromamba/envs/gigapath/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 419, in _get_iterator
    return _SingleProcessDataLoaderIter(self)
  File "/public/home/jijh/micromamba/envs/gigapath/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 754, in __init__
    self._dataset_fetcher = _DatasetKind.create_fetcher(
                            ~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        self._dataset_kind,
        ^^^^^^^^^^^^^^^^^^^
    ...<3 lines>...
        self._drop_last,
        ^^^^^^^^^^^^^^^^
    )
    ^
  File "/public/home/jijh/micromamba/envs/gigapath/lib/py

# 修复错误

In [14]:
# 单元格 3: 复制并粘贴我们正在测试的核心代码 (最终修复版)

import logging
import scanpy as sc
import webdataset as wds
import torch
import torch.utils.data
from torch_geometric.data import Data as PyGData, Batch as PyGBatch
from PIL import Image
import io

# -------------------------------------------------------------
# 这是您计划放在 src/open_clip_train/spaglam_data.py 的代码
# -------------------------------------------------------------
class SpaGLaMWDataset(torch.utils.data.IterableDataset):
    def __init__(self, tar_urls, anndata_path, image_processor, tokenizer):
        super().__init__()
        self.tar_urls = tar_urls
        self.anndata_path = anndata_path
        self.image_processor = image_processor
        self.tokenizer = tokenizer
        self.adata = None

    def _init_worker(self):
        if self.adata is None:
            self.adata = sc.read_h5ad(self.anndata_path, backed='r')

    def _process_sample(self, sample):
        self._init_worker()
        center_key = sample['__key__']
        
        try:
            # 1. 解码中心点数据 (现在更加稳健)
            center_img = sample['png'] # wds.decode("pilrgb") 保证了这是一个PIL Image
            
            # ------------------- 文本解码最终修复点 -------------------
            # 防御性编程：检查类型，因为webdataset可能已经自动解码了.txt文件
            txt_data = sample['txt']
            if isinstance(txt_data, bytes):
                center_sent = txt_data.decode('utf-8')
            else:
                center_sent = txt_data  # 它已经是字符串了
            # ---------------------------------------------------------

            # 2. 从 AnnData 获取邻居信息
            center_idx = self.adata.obs_names.get_loc(center_key)
            neighbor_indices = self.adata.obsp['spatial_connectivities'][center_idx].indices
            neighbor_keys = self.adata.obs_names[neighbor_indices].tolist()
            
            all_keys = [center_key] + neighbor_keys
            num_nodes = len(all_keys)
            
            paths_df = self.adata.obs.loc[all_keys, ['image_path', 'sentence_path']]
            
            # 3. 加载所有节点的数据并预处理
            images_processed, sentences = [], []
            for key in all_keys:
                if key == center_key:
                    img = center_img
                    sent = center_sent
                else:
                    img_path, sent_path = paths_df.loc[key]
                    try: img = Image.open(img_path).convert("RGB")
                    except (FileNotFoundError, OSError): img = Image.new('RGB', (224, 224), color='grey')
                    try: 
                        with open(sent_path, 'r') as f: sent = f.read()
                    except (FileNotFoundError, OSError): sent = ""
                
                images_processed.append(self.image_processor(img))
                sentences.append(sent)

            tokenized_texts = self.tokenizer(sentences)
            
            # 4. 构建局部图结构
            key_to_local_idx = {key: i for i, key in enumerate(all_keys)}
            edge_list = []
            
            for i, key in enumerate(all_keys):
                global_idx = self.adata.obs_names.get_loc(key)
                row_indices = self.adata.obsp['spatial_connectivities'][global_idx].indices
                
                for neighbor_global_idx in row_indices:
                    neighbor_key = self.adata.obs_names[neighbor_global_idx]
                    if neighbor_key in key_to_local_idx:
                        j = key_to_local_idx[neighbor_key]
                        if i < j:
                            edge_list.append([i, j])
                            edge_list.append([j, i])

            edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous() if edge_list else torch.empty((2, 0), dtype=torch.long)
            
            # 5. 返回PyG Data对象
            return PyGData(
                x_image=torch.stack(images_processed),
                x_text=tokenized_texts,
                edge_index=edge_index,
                num_nodes=num_nodes,
                center_key=center_key,
            )
        except Exception as e:
            logging.error(f"Error processing sample {center_key}: {e}", exc_info=True)
            return None

    def __iter__(self):
        pipeline = wds.DataPipeline(
            wds.shardlists.SimpleShardList(self.tar_urls),
            wds.split_by_worker,
            wds.tarfile_to_samples(),
            wds.decode("pilrgb"), # 只解码图像，文本保持原样
            wds.map(self._process_sample),
            wds.select(lambda x: x is not None),
        )
        return iter(pipeline)

# -------------------------------------------------------------
# 其他代码 (get_spaglam_dataset, get_dataset_fn, get_data) 保持不变
# -------------------------------------------------------------

def get_spaglam_dataset(args, preprocess_fn, is_train, epoch=0, tokenizer=None):
    # ... (此函数无需修改)
    input_shards = args.train_data
    dataset = SpaGLaMWDataset(
        tar_urls=input_shards,
        anndata_path=args.anndata_path,
        image_processor=preprocess_fn,
        tokenizer=tokenizer,
    )
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=args.batch_size,
        num_workers=args.workers,
        collate_fn=PyGBatch.from_data_list,
        drop_last=is_train,
    )
    dataloader.num_samples = args.train_num_samples
    if dataloader.num_samples:
        dataloader.num_batches = dataloader.num_samples // args.batch_size
    else:
        dataloader.num_batches = 0
    return DataInfo(dataloader=dataloader)

def get_dataset_fn(data_path, dataset_type):
    # ... (此函数无需修改)
    if dataset_type == "spaglam":
        return get_spaglam_dataset
    else:
        raise ValueError(f"Unsupported dataset type for this test: {dataset_type}")

def get_data(args, preprocess_fns, epoch=0, tokenizer=None):
    # ... (此函数无需修改)
    preprocess_train, preprocess_val = preprocess_fns
    data = {}
    if args.train_data:
        data["train"] = get_dataset_fn(args.train_data, args.dataset_type)(
            args, preprocess_train, is_train=True, epoch=epoch, tokenizer=tokenizer)
    return data

print("✅ 核心数据处理代码已加载到Notebook中 (最终修复版)。")

✅ 核心数据处理代码已加载到Notebook中 (最终修复版)。


In [15]:
# 单元格 4: 模拟训练脚本的调用并验证输出

# --- 1. 模拟命令行参数 (args) ---
from argparse import Namespace
args = Namespace(
    dataset_type='spaglam',
    train_data=[DUMMY_SHARD_PATH_PATTERN % 0], # 使用我们创建的模拟分片
    anndata_path=DUMMY_ADATA_PATH,            # 使用我们创建的模拟AnnData
    train_num_samples=NUM_SPOTS,
    batch_size=4,
    workers=0, # 关键：在notebook中设为0，以便于调试，避免多进程问题
    world_size=1,
    distributed=False
)
print("--- 模拟参数 ---")
for k, v in vars(args).items():
    print(f"{k}: {v}")

# --- 2. 模拟预处理器和分词器 ---
from torchvision import transforms
dummy_preprocess_fn = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 一个非常简单的分词器，返回固定长度的随机整数张量
dummy_tokenizer = lambda texts: torch.randint(0, 49408, (len(texts), 77), dtype=torch.long)
print("\n✅ 模拟的预处理器和分词器已准备就绪。")


# --- 3. 调用 get_data 获取 DataLoader ---
print("\n--- 调用 get_data() ---")
data = get_data(args, (dummy_preprocess_fn, None), tokenizer=dummy_tokenizer)
dataloader = data['train'].dataloader
print("✅ 成功创建DataLoader！")

# --- 4. 从DataLoader中取出一个批次并进行最终验证 ---
print("\n--- 最终验证：从DataLoader中获取一个批次 ---")
try:
    batch = next(iter(dataloader))
    
    print("\n✅ [成功] 成功从DataLoader中获取一个批次的数据！")
    print("-" * 50)
    
    # --- 验证 1: 检查批次类型 ---
    print(f"批次类型: {type(batch)}")
    assert isinstance(batch, PyGBatch), "批次类型不正确！应为 torch_geometric.data.Batch"
    print("✅ [类型正确] 批次是 torch_geometric.data.Batch 对象。")
    
    # --- 验证 2: 检查批次内容 ---
    print("\n批次内容概览:")
    print(batch)
    
    # --- 验证 3: 检查关键属性 ---
    print("\n关键属性检查:")
    print(f"批次中的图数量 (应等于batch_size): {batch.num_graphs}")
    assert batch.num_graphs == args.batch_size, "批次中的图数量不等于batch_size"
    
    total_nodes = sum([NUM_NEIGHBORS + 1] * args.batch_size) # 理论上的节点总数
    print(f"批次中的节点总数: {batch.num_nodes}")
    # 注意：由于某些邻居可能缺失，实际节点数可能略小于理论值，这里不作严格断言
    
    print(f"x_image 张量形状: {batch.x_image.shape}")
    assert batch.x_image.shape[0] == batch.num_nodes
    assert tuple(batch.x_image.shape[1:]) == (3, IMAGE_SIZE[0], IMAGE_SIZE[1])
    
    print(f"x_text 张量形状: {batch.x_text.shape}")
    assert batch.x_text.shape[0] == batch.num_nodes
    
    print(f"edge_index 张量形状: {batch.edge_index.shape}")
    
    print(f"batch 索引向量长度 (应等于节点总数): {len(batch.batch)}")
    assert len(batch.batch) == batch.num_nodes
    
    print("\n✅ [内容正确] 批次对象的所有关键属性都符合预期！")
    print("-" * 50)
    print("\n🎉🎉🎉 恭喜！数据加载管道验证成功！您可以放心地将这些代码集成到open-clip项目中了。🎉🎉🎉")

except Exception as e:
    print(f"\n❌ [失败] 在获取或验证批次时发生错误: {e}")
    import traceback
    traceback.print_exc()

--- 模拟参数 ---
dataset_type: spaglam
train_data: ['/cwStorage/nodecw_group/jijh/openclip_train/spaglam_test_environment/shards/shard-000000.tar']
anndata_path: /cwStorage/nodecw_group/jijh/openclip_train/spaglam_test_environment/metadata/dummy_master_adata.h5ad
train_num_samples: 100
batch_size: 4
workers: 0
world_size: 1
distributed: False

✅ 模拟的预处理器和分词器已准备就绪。

--- 调用 get_data() ---
✅ 成功创建DataLoader！

--- 最终验证：从DataLoader中获取一个批次 ---

✅ [成功] 成功从DataLoader中获取一个批次的数据！
--------------------------------------------------
批次类型: <class 'abc.DataBatch'>
✅ [类型正确] 批次是 torch_geometric.data.Batch 对象。

批次内容概览:
DataBatch(edge_index=[2, 106], x_image=[28, 3, 224, 224], x_text=[28, 77], num_nodes=28, center_key=[4], batch=[28], ptr=[5])

关键属性检查:
批次中的图数量 (应等于batch_size): 4
批次中的节点总数: 28
x_image 张量形状: torch.Size([28, 3, 224, 224])
x_text 张量形状: torch.Size([28, 77])
edge_index 张量形状: torch.Size([2, 106])
batch 索引向量长度 (应等于节点总数): 28

✅ [内容正确] 批次对象的所有关键属性都符合预期！
--------------------------------------------------
